In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import gray2rgb, rgb2gray, label2rgb

In [2]:
import numpy as np
from scipy import sparse
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, roc_auc_score

sys.path.append("..")
from inverse_hvp import inverse_hvp_lr_newtonCG
import argparse
import time
import pdb
import os
np.random.seed(0)

In [3]:
# select the dataset used
dataset_name = "mnist"
# regularization parameter for Logistic Regression
C = 0.1

In [4]:

# tool box
def load_mnist(validation_size = 5000):
    import gzip
    def _read32(bytestream):
        dt = np.dtype(np.uint32).newbyteorder(">")
        return np.frombuffer(bytestream.read(4),dtype=dt)[0]

    def extract_images(f):
        print("Extracting",f.name)
        with gzip.GzipFile(fileobj=f) as bytestream:
            magic = _read32(bytestream)
            num_images = _read32(bytestream)
            rows = _read32(bytestream)
            cols = _read32(bytestream)
            buf = bytestream.read(rows * cols * num_images)
            data = np.frombuffer(buf,dtype=np.uint8)
            data = data.reshape(num_images,rows,cols,1)
            return data
    
    def extract_labels(f):
        print('Extracting', f.name)
        with gzip.GzipFile(fileobj=f) as bytestream:
            magic = _read32(bytestream)
            num_items = _read32(bytestream)
            buf = bytestream.read(num_items)
            labels = np.frombuffer(buf, dtype=np.uint8)
            return labels

    #data_dir = "./data"
    data_dir = ""
    TRAIN_IMAGES = os.path.join(data_dir,'train-images-idx3-ubyte.gz')
    with open(TRAIN_IMAGES,"rb") as f:
        train_images = extract_images(f)

    TRAIN_LABELS =  os.path.join(data_dir,'train-labels-idx1-ubyte.gz')
    with open(TRAIN_LABELS,"rb") as f:
        train_labels = extract_labels(f)

    TEST_IMAGES =  os.path.join(data_dir,'t10k-images-idx3-ubyte.gz')
    with open(TEST_IMAGES,"rb") as f:
        test_images = extract_images(f)

    TEST_LABELS =  os.path.join(data_dir,'t10k-labels-idx1-ubyte.gz')
    with open(TEST_LABELS,"rb") as f:
        test_labels = extract_labels(f)

    # split train and val
    train_images = train_images[validation_size:]
    train_labels = train_labels[validation_size:]

    # preprocessing
    train_images = train_images.astype(np.float32) / 255
    test_images  = test_images.astype(np.float32) / 255
    
    # reshape for logistic regression
    train_images = np.reshape(train_images, [train_images.shape[0], -1])
    test_images = np.reshape(test_images, [test_images.shape[0], -1])
    return train_images,train_labels,test_images,test_labels

def load_data_two_class(dataset_name,va_ratio):
  x_train,y_train,x_test,y_test = load_mnist()
  pos_class = 1
  neg_class = 7
  x_train,y_train = filter_dataset(x_train,y_train,pos_class,neg_class)
  x_va,y_va = filter_dataset(x_test,y_test,pos_class,neg_class)
  y_va = y_va.astype(int)
  y_train = y_train.astype(int)

  num_va_sample = int((1-va_ratio) * x_train.shape[0])
  x_val = x_train[num_va_sample:]
  y_val = y_train[num_va_sample:]
  x_train = x_train[:num_va_sample]
  y_train = y_train[:num_va_sample]
  x_te = x_va
  y_te = y_va

  return x_train,y_train,x_val,y_val,x_te,y_te


def filter_dataset(X, Y, pos_class, neg_class, mode=None):
    
    """
    Filters out elements of X and Y that aren't one of pos_class or neg_class
    then transforms labels of Y so that +1 = pos_class, -1 = neg_class.

    They are all 10-classes image classification data sets while Logistic regression can only handle binary classification. we
    select the number 1 and 7 as positive and negative classes;
    """

    assert(X.shape[0] == Y.shape[0])
    assert(len(Y.shape) == 1)

    Y = Y.astype(int)
    
    pos_idx = Y == pos_class
    neg_idx = Y == neg_class        
    Y[pos_idx] = 1
    Y[neg_idx] = -1
    idx_to_keep = pos_idx | neg_idx
    X = X[idx_to_keep, ...]
    Y = Y[idx_to_keep]
    if Y.min() == -1 and mode != "svm":
        Y = (Y + 1) / 2
        Y.astype(int)
    return (X, Y)

# load data, pick 30% as the Va set
x_train,y_train,x_va,y_va,x_te,y_te = load_data_two_class(dataset_name,va_ratio=0.3)



print("x_train, nr sample {}, nr feature {}".format(x_train.shape[0],x_train.shape[1]))
print("x_va,    nr sample {}, nr feature {}".format(x_va.shape[0],x_va.shape[1]))
print("x_te,    nr sample {}, nr feature {}".format(x_te.shape[0],x_te.shape[1]))
print("Tr: Pos {} Neg {}".format(y_train[y_train==1].shape[0],y_train[y_train==0].shape[0]))
print("Va: Pos {} Neg {}".format(y_va[y_va==1].shape[0],y_va[y_va==0].shape[0]))
print("Te: Pos {} Neg {}".format(y_te[y_te==1].shape[0],y_te[y_te==0].shape[0]))

Extracting train-images-idx3-ubyte.gz
Extracting train-labels-idx1-ubyte.gz
Extracting t10k-images-idx3-ubyte.gz
Extracting t10k-labels-idx1-ubyte.gz
x_train, nr sample 8325, nr feature 784
x_va,    nr sample 3569, nr feature 784
x_te,    nr sample 2163, nr feature 784
Tr: Pos 4395 Neg 3930
Va: Pos 1784 Neg 1785
Te: Pos 1135 Neg 1028


In [5]:
num_tr_sample = x_train.shape[0]
sample_ratio = 0.6
obj_sample_size = int(sample_ratio * num_tr_sample)


"""
# Our unweighted method can downweight the bad cases which cause high test loss to the our model, which is an important reason of its ability to improve result with less data.
To show the performance of our methods in noisy label situation, we perform addtional experiments with some training
labels being flipped. The results show the enlarging superiority of our subsampling methods 
"""

# flip labels
idxs = np.arange(y_train.shape[0])
flip_ratio = 0.4
np.random.shuffle(idxs)
num_flip = int(flip_ratio * len(idxs))
y_train[idxs[:num_flip]] = np.logical_xor(np.ones(num_flip), y_train[idxs[:num_flip]]).astype(int)

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer

class PipeStep(object):
    """
    Wrapper for turning functions into pipeline transforms (no-fitting)
    """
    def __init__(self, step_func):
        self._step_func=step_func
    def fit(self,*args):
        return self
    def transform(self,X):
        return self._step_func(X)


makegray_step = PipeStep(lambda img_list: [rgb2gray(img) for img in img_list])
flatten_step = PipeStep(lambda img_list: [img.ravel() for img in img_list])

simple_rf_pipeline = Pipeline([
    ('Make Gray', makegray_step),
    ('Flatten Image', flatten_step),
    #('Normalize', Normalizer()),
    #('PCA', PCA(16)),
    ('RF', RandomForestClassifier())
                              ])

In [12]:
simple_rf_pipeline.fit(x_train, y_train)

/tmp/ipykernel_158689/3157965683.py:17: FutureWarning: Non RGB image conversion is now deprecated. For RGBA images, please use rgb2gray(rgba2rgb(rgb)) instead. In version 0.19, a ValueError will be raised if input image last dimension length is not 3.
  makegray_step = PipeStep(lambda img_list: [rgb2gray(img) for img in img_list])


Pipeline(steps=[('Make Gray', <__main__.PipeStep object at 0x7f7675729b50>),
                ('Flatten Image', <__main__.PipeStep object at 0x7f7675729c10>),
                ('RF', RandomForestClassifier())])